## ಅಗತ್ಯ ಲೈಬ್ರರಿಗಳನ್ನು ಆಮದು ಮಾಡಿ
Phi-4 ಮಾದರಿಯನ್ನು ಲೋಡ್ ಮತ್ತು ಬಳಸಲು ಅಗತ್ಯವಿರುವ PyTorch ಮತ್ತು transformers ಲೈಬ್ರರಿಗಳನ್ನು ಆಮದು ಮಾಡಿ.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## ಯാദೃಚ್ಛಿಕ ಸೀಡ್ ಅನ್ನು ಹೊಂದಿಸಿ

ಯಾದೃಚ್ಛಿಕ ಸೀಡ್ ಅನ್ನು ಹೊಂದಿಸಿ, ವಿಭಿನ್ನ ರನ್‌ಗಳಲ್ಲಿ ಪುನರಾವರ್ತಿಸಬಹುದಾದ ಫಲಿತಾಂಶಗಳನ್ನು ಖಾತ್ರಿ ಮಾಡಲು.


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoning ಮಾದರಿ ಮತ್ತು ಟೋಕನೈಜರ್ ಅನ್ನು ಲೋಡ್ ಮಾಡಿ
Microsoft Phi-4-mini-flash-reasoningmodel ಮತ್ತು ಅದರ ಅನುಗುಣವಾದ ಟೋಕನೈಜರ್ ಅನ್ನು Hugging Face ನಿಂದ ಲೋಡ್ ಮಾಡಿ. ಮಾದರಿಯನ್ನು ವೇಗವಾದ ಇನ್ಫರೆನ್ಸ್‌ಗೆ CUDA ಮೇಲೆ ಲೋಡ್ ಮಾಡಲಾಗುತ್ತದೆ.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Prepare Input Message
ದ್ವಿತೀಯ ಸಮೀಕರಣದ ಗಣಿತ ಸಮಸ್ಯೆಯನ್ನು ಹೊಂದಿರುವ ಸಂವಾದ ಸಂದೇಶವನ್ನು ರಚಿಸಿ ಮತ್ತು ಮಾದರಿಗಾಗಿ ಚಾಟ್ ಟೆಂಪ್ಲೇಟ್ ಬಳಸಿ ಅದನ್ನು ರೂಪಬದ್ಧಗೊಳಿಸಿ.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## ಪ್ರತಿಕ್ರಿಯೆ ರಚಿಸಿ
ಮಾದರಿಯಿಂದ temperature ಮತ್ತು top_p ಇತ್ಯಾದಿ ನಿರ್ದಿಷ್ಟ ಪ್ಯಾರಾಮೀಟರ್‌ಗಳನ್ನು ಬಳಸಿಕೊಂಡು ಔಟ್‌ಪುಟ್‌ನಲ್ಲಿನ ನಿಯಂತ್ರಿತ ಯಾದೃಚ್ಛಿಕತಿಗೆ ಪ್ರತಿಕ್ರಿಯೆ ರಚಿಸಿ.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## ಔಟ್‌ಪುಟ್ ಅನ್ನು ಪಠ್ಯಕ್ಕೆ ಡಿಕೋಡ್ ಮಾಡಿ
ಉತ್ಪತ್ತಿಯಾದ ಟೋಕನ್ ಕ್ರಮಗಳನ್ನು ಮನುಷ್ಯನ ಓದಬಹುದಾದ ಪಠ್ಯವಾಗಿ ಪರಿವರ್ತಿಸಿ, ಮೂಲ ಇನ್‌ಪುಟ್ ಟೋಕನ್‌ಗಳನ್ನು ಹೊರತುಪಡಿಸಿ ಕೇವಲ ಮಾದರಿಯ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ಮಾತ್ರ ತೋರಿಸಿ.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ನಿರಾಕರಣೆ:
ಈ ದಸ್ತಾವೇಜನ್ನು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಶುದ್ಧತೆಗೆ ಪ್ರಯತ್ನಿಸಿದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸತ್ಯತೆಯುಗಳಿರಬಹುದು ಎಂಬುದನ್ನು ದಯವಿಟ್ಟು ಗಮನದಲ್ಲಿರಿಸಿ. ಮೂಲಭಾಷೆಯಲ್ಲಿ ಇರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಗಂಭೀರ ಮಾಹಿತಿಗೆ ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗಿದೆ. ಈ ಅನುವಾದವನ್ನು ಬಳಸದರಿಂದ ಉಂಟಾಗುವ ಯಾವುದಾದರೂ ತಪ್ಪು ಅರ್ಥಗೊಳ್ಳುವಿಕೆಗಳು ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಜವಾಬ್ದಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
